<a href="https://colab.research.google.com/github/PhilipRuebeling/master_thesis/blob/main/photonic_devices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}\newcommand{\bra}[1]{\left\langle
{#1}\right|}$$

In [ ]:
%pip install fst-pso  # particle swarm optimization

In [ ]:
import numpy as np
import fstpso as fp

import scipy.special


# Electro-optic modulator (EOM)


The phase modulation of an [electro-optic modulator](https://en.wikipedia.org/wiki/Electro-optic_modulator) is described by its driving voltage

$a(t) = \sum_{k=1}^{m} a_{k}\sin(\Omega_{k} t + \varphi_{k})$

Here we call $m = 1, 2, \dots$ the modulation depth. When we concentrate on single-tone modulation we will set this to be equal to $1$.

$a^{\dagger}_{r} \rightarrow Sa^{\dagger}_{r}S^{\dagger} = \sum_{q} (-ie^{-i\vartheta})^{q} J_{q}(m)a^{\dagger}_{r+q}$.

In [ ]:
global mod_depth = 1     # modulation depth
global a_min = 00.00     # minimal modulation amplitude
global a_max = 10.00     # maximal modulation amplitude

This defines the unitary in time domain (TD).

In [ ]:
def unitaryTD(parameters, mod_depth, n_bins) :
    U = np.zeros(shape=(n_bins, n_bins), dtype="complex64")
    amplitude, phase = parameters
    for i in range(0, d):
        U[i][i] += np.exp(1j*(amplitude*np.sin(np.pi*i/d + phase)))
    return U

In [ ]:
# testing if unitaryTD returns a unitary matrix
U = np.around(unitaryTD(parameters, mod_depth, n_bins), decimals=9)
is_unitary = np.allclose(np.eye(n_bins), U @ RT.adjoint(U))
print('unitaryTD() returns a unitary matrix: ' + str(is_unitary))

Plotting the modulation signal

In [ ]:
def plot_modulation(parameters, mod_depth, n_bins) :
    amplitude, phase = parameters
    t = np.arange(0, 10*n_bins)
    s = np.around(amplitude*np.sin(np.pi*t/n_bins + phase), decimals=9)
    plt.title('modulation signal ot the EOM in TD')
    plt.plot(t/n_bins, s)
    plt.xlabel('time [norm.]')
    plt.ylabel('driving voltage [arb.]')
    plt.show()
    plt.close()
    pass

In [ ]:
plot_modulation(parameters, mod_depth, n_bins)

Here is the corresponding unitary in frequency domain (FD).

In [ ]:
def unitaryFD(parameters, mod_depth, n_bins) :
    F = dft(n_bins)/np.sqrt(n_bins)
    U = unitaryTD(parameters, mod_depth, n_bins) #doesn't involve yet.
    return F @ U @ RT.adjoint(F)

In [ ]:
# testing if unitaryFD returns a unitary matrix
U = np.around(unitaryFD(parameters, mod_depth, n_bins), decimals=9)
is_unitary = np.allclose(np.eye(n_bins), U @ RT.adjoint(U))
print('unitaryFD() returns a unitary matrix: ' + str(is_unitary))

In [ ]:
# this function will not return some unitary matrix, 
# since we truncate the hilbert space
def besselFD(parameters, n_bins, f=0.42) :
    U = np.zeros(shape=(n_bins, n_bins), dtype='complex64')
    amplitude, phase = parameters
    for i in range(0, d) :
        for j in range(0, d) :
            if(np.abs(i-j) <= math.ceil(amplitude+2)) :
                U[i][j] = special.jv(i-j, f*amplitude)*(-1j*np.exp(-1j*phase))**(i-j)
    return U

# Phase Shaper (PS)



TODO: explain how the PS works.

In [ ]:
def unitaryFD(parameters, n_bins) :
    U = np.array(np.zeros(shape=(n_bins, n_bins)), dtype='complex64')
    for i in range(0, n_bins):
        U[i][i] = np.exp(1j*theta[i])
    return U

In [ ]:
# testing if unitaryFD returns a unitary matrix
U = np.around(unitaryFD(parameters, n_bins), decimals=9)
is_unitary = np.allclose(np.eye(n_bins), U @ RT.adjoint(U))
print('unitaryFD() returns a unitary matrix: ' + str(is_unitary))

# Classical Frequency Processor (CFP)

Before we consider the quantum case, we shall examine a frequency processor that operates with classical intensities. Just like in the quantum case we can have "classical" interference between the different frequency bins which is due to the frequncy mixing of an EOM. Therefore it seems an interesting question, if we can perform certain deep learning tasks on a CFP just as good as on a QFP.

Of course it is possible to describe a CFP with classical fields. Here we choose the same quantum framework as we previously encoutered for the QFP to analyse the CFP. This is possbile if we choose as input states of light that are sufficiently "classical" with relativley high intensities. Thus coherent states are a quite natural choice, as they are essentially "laser light". 

Of course we could choose other states, epecially mixed states, where we no longer operate in the regime of "pure" coherence. For such states, we expect the interference to become negible. Thus no photon interactions are possible and we expect the CFP to perform bad in deep learning tasks, due to the lack of flexibility of the underlying stochastic model.

Let's get somewhat techncial. A coherent state $\ket{\alpha} = D(\alpha)\ket{vac}$ can be described by applying the displacement operator, which is defined by $D(\alpha) = \exp (\sum_{r=0}^{M-1} \alpha_{r} a^{\dagger}_{r}+ \alpha_{r}^{*} a_{r})$, to the vacuum. We shall assume, that our CFP is fed with an "classical" input state, where each frequency bin is uniformly populated by some intensity proportional to $|\alpha|^{2}$. This input state may be written as $\ket{in} = \ket{\alpha}_{0} \otimes \dots \otimes \ket{\alpha}_{M-1}$.

In [ ]:
global n_layers, n_bins = 4, 32  # defines the architecture of the CFP

In [ ]:
def unitaryV(parameters_ps, parameters_em) :
    V = np.eye(bins, dtype='complex64')
    for layer in range(0, n_layers) :
        U_em = em.unitaryFD(parameters_em[layer], mod_depth, n_bins)
        U_ps = ps.unitaryFD(parameters_ps[layer], n_bins)
        V = V @ U_ps @ U_eom
    return V

In [ ]:
def classical_input() :
  

# Quantum Frequency Processor (QFP)

We might investigate the HOM interference a little bit closer. What happens, if the photons become distinguishable, e.g. by their polarization. It is well known from the beam splitter interference experiment performed by [Hong, Ou and Mandel](https://en.wikipedia.org/wiki/Hong%E2%80%93Ou%E2%80%93Mandel_effect) (1987), that 

# Benchmarking with BAS22

BAS patterns


In [ ]:
def sample_bas22() :
    bas = np.zeros(d)
    bas[0], bas[3], bas[5], bas[10], bas[12], bas[15] = 1, 1, 1, 1, 1, 1
    return bas/6

# Optimization

In [ ]:
swarm_size = 100    # number of particles used in the PSO
iterations = 100    # number of iterations in the PSO 

In [ ]:
def optimize_circuit_fuzzypso(cost_function, search_space) :
  FP = fp.FuzzyPSO()
  FP.set_search_space(search_space)
  FP.set_fitness(cost_function)  
  FP.set_swarm_size(swarm_size)
  best_solution, fitness = FP.solve_with_fstpso(max_iter=iterations)
  return [best_solution, fitness]

In [ ]:
def setupSearchSpace(a_min, a_max) :
    phase_range, amplitude_range = [0, 2*np.pi], [a_min, a_max]
    search_space = []
    for i in range(0, layers*n_bins) :
        search_space.append(phase_range)
    for i in range(0, layers*mod_depth) :
        search_space.append(amplitude_range)
        search_space.append(phase_range)
    return search_space

In [ ]:
search_space = fp.setupSearchSpace(a_min, a_max)

# Evaluation